Big Data ETL of Amazon Watch Reviews

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-20 21:23:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-02-20 21:23:13 (5.97 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watches_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
watches_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

# Size of Data

In [ ]:
watches_df.count()

960872

# Cleaned up DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = watches_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370| 2015-08-31|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722| 2015-08-31|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390| 2015-08-31|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625| 2015-08-31|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221| 2015-08-31|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695| 2015-08-31|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279| 2015-08-31|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113| 2015-08-31|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892| 2015-08-31|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407| 2015-08-31|
|R15C7QEZT0LGZN|    9962330|B00KGTVGKS|      28017857| 2015-08-31|
|R361XSS37V0NCZ|   16097204|B0039UT5OU|     685450910| 2015-08

In [ ]:
# drop the duplicates
products_df = watches_df.select(["product_id", "product_title"]).drop_duplicates()

In [ ]:
# create review df
review_df = watches_df.select(['review_id', 'review_headline', 'review_body'])
review_df.show()

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R3O9SGZBVQBV76|          Five Stars|Absolutely love t...|
| RKH8BNC3L5DLF|I love thiswatch ...|I love this watch...|
|R2HLE8WKZSU3NL|           Two Stars|           Scratches|
|R31U3UH5AZ42LL|          Five Stars|It works well on ...|
|R2SV659OUJ945Y|Beautiful face, b...|Beautiful watch f...|
| RA51CP8TR5A2L|       No complaints|i love this watch...|
| RB2Q7DLDN6TH6|          Five Stars|for my wife and s...|
|R2RHFJV0UYBK3Y|I was about to bu...|I was about to bu...|
|R2Z6JOQ94LFHEP|      Perfect watch!|Watch is perfect....|
| RX27XIIWY5JPB|Great quality and...|Great quality and...|
|R15C7QEZT0LGZN|           Satisfied|The watch was pre...|
|R361XSS37V0NCZ|I do not think th...|I bought this wat...|
| ROTNLALUAJAUB|         Three Stars|It is a cheap wat...|
|R2DYX7QU6BGOHR|          Five Stars|Heavier than i th..

In [ ]:
# group customer id by customer count
customers_df = watches_df.groupby('customer_id').count().withColumnRenamed('count', 'customer_count')
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1567510|             1|
|   19502021|             1|
|   12819130|             1|
|   35329257|             2|
|     108460|             1|
|    5453476|             1|
|   29913055|             1|
|   30717305|             1|
|    1570030|             1|
|   19032020|             1|
|   44178035|             1|
|   26079415|             2|
|   14230926|             1|
|   43478048|             2|
|   43694941|             1|
|   12318815|             3|
|   13731855|             1|
|     740134|             1|
|   41956754|             1|
|   20324070|             3|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# select columns from vine_df
vine_df = watches_df.select(['review_id', 'star_rating', 'helpful_votes', 'vine'])
vine_df.show()

+--------------+-----------+-------------+----+
|     review_id|star_rating|helpful_votes|vine|
+--------------+-----------+-------------+----+
|R3O9SGZBVQBV76|          5|            0|   N|
| RKH8BNC3L5DLF|          5|            0|   N|
|R2HLE8WKZSU3NL|          2|            1|   N|
|R31U3UH5AZ42LL|          5|            0|   N|
|R2SV659OUJ945Y|          4|            0|   N|
| RA51CP8TR5A2L|          5|            0|   N|
| RB2Q7DLDN6TH6|          5|            1|   N|
|R2RHFJV0UYBK3Y|          1|            1|   N|
|R2Z6JOQ94LFHEP|          5|            1|   N|
| RX27XIIWY5JPB|          4|            0|   N|
|R15C7QEZT0LGZN|          4|            2|   N|
|R361XSS37V0NCZ|          1|            0|   N|
| ROTNLALUAJAUB|          3|            0|   N|
|R2DYX7QU6BGOHR|          5|            0|   N|
| RWASY7FKI7QOT|          5|            2|   N|
|R2KKYZIN3CCL21|          5|            0|   N|
|R22H4FGVD5O52O|          4|            1|   N|
|R11UACZERCM4ZY|          5|            

# Push to AWS RDS instance

In [ ]:
# set up properties to connect to AWS Postgres
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.c3i5uoorx78s.us-east-1.rds.amazonaws.com:5432/l1watchdb"
config = {"user":"postgres", "password": "<redacted>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)